###方式1：LLMs自行生成

In [1]:
#3.将原始文本和本地知识库建立faiss搜索，得到topk相关doc
# coding:utf-8
# 导入必备的工具包
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

# 设置工作目录
os.chdir("D:/研究生论文/小论文2/代码")

# 加载FAISS向量库
EMBEDDING_MODEL = './02RAG_质量特征匹配阶段/RAG代码/embedding model/m3e-base'
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
db = FAISS.load_local('D:/faiss/microdramas', embeddings, allow_dangerous_deserialization=True)   #D:/faiss

def get_related_content(related_docs):
    related_content = []
    for doc in related_docs:
        related_content.append(doc.page_content.replace('\n\n', '\n'))
    return '\n'.join(related_content)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15104\77445456.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
d:\software\anaconda\Anaconda_envs\envs\py39RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

OPENAI_API_KEY='sk-OrvIqdniDCI3LbtO0p6cSYuN0x0zo5j1jrEFQWyNEuIridSc'
OPENAI_API_BASE='https://api.gpts.vin/v1'  #https://api.chatanywhere.com.cn/v1
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",openai_api_key=OPENAI_API_KEY, openai_api_base=OPENAI_API_BASE)#gpt-3.5-turbo

In [ ]:
def define_prompt(related_docs):
    
    # 构建模板
    PROMPT_TEMPLATE = """
          你将收到一组微短剧的在线评论文档。你的任务是:识别每个文档中涉及的质量特征(最多3个);然后,进行情感分析,判断情感极性（积极/中立/消极）并计算情感得分；最后,以指定格式输出每个文档对应的质量特征标签、情感级性和情感得分。

          现有的质量特征标签为：文化深度、剧情合理性、叙事节奏、角色表现、动作场面呈现、服化道匹配度、台词质量、音效与配乐。其具体含义如下：
          文化深度：微短剧是否具备思想性和文化价值，能否引发观众共鸣或思考，并传递积极的社会价值观。
          剧情合理性：剧情逻辑是否严谨，故事情节是否连贯、完整，角色行为是否符合设定。
          叙事节奏：故事节奏是否流畅，是否存在冗长或过快的节奏变化影响观感。
          角色表现：演员表演是否真实自然，能否准确表达角色的个性、情感与成长。
          动作场面呈现：动作场面的特效表现是否真实，武术指导是否到位，是否增强视觉冲击力。
          服化道匹配度：服装、化妆和道具是否符合角色设定、时代背景和剧情需求，是否增强角色表现力。
          台词质量：台词是否符合角色设定，语言表达是否自然流畅，避免刻意生硬。
          音效与配乐：背景音乐、音效是否恰当，能否有效增强剧情氛围和情感表达。

          情感级性包括：积极、中立和消极。积极对应的情感得分取值范围为(0,1],中立为0,消极为[-1,0)。
            
          
          [示例]
          示例 1:将"叙事节奏"指定给文档，并给出质量特征标签对应的情感级性、情感得分以及置信度
          文档：
          剧情拖沓，主角半天才说一句话，看得人着急。
          你的回答：
          叙事节奏|消极|-0.70:0.85


          示例 2:将"角色表现","台词质量"指定给文档，并给出质量特征标签对应的情感级性、情感得分以及置信度
          文档：
          演技好,台词有感染力。
          你的回答：
          角色表现|积极|0.85:0.90,台词质量|积极|0.90:0.88

          示例 3:文档无质量特征
          文档：
          @肖yao 你在这看到10集。我在@你别的。
          你的回答：
          None

          [已知内容]:
          {context}

          [规则]
          -质量特征必须存在于给出的质量特征列表中，不可以在添加新的质量特征。
          -每个文档对应的质量特征至多3个,如果识别出的质量特征大于3个,则只输出最相关的3个质量特征。
          -如果文档中不包含情感词,则其情感得分为0,情感级性为中立。
          -只有当质量特征识别的置信度≥0.7时，才输出该质量特征。
          -输出格式中在情感得分后添加置信度，用冒号分隔。

          [命令]
          -严格按照示例格式输出结果，不要输出其他多余内容！
          -将文档分配给最相关的质量特征，输出文档中的质量特征标签、情感级性、情感得分和置信度。
          -只输出置信度≥0.7的质量特征。
          

           
           """
    
   
    prompt = PROMPT_TEMPLATE.format(context=related_docs)

    print(f'my_prompt-->{prompt}')
    return prompt

In [4]:
import datetime
import pandas as pd

data = pd.read_excel('./01数据预处理/02输出结果-预处理/桃花马上请长缨/7.最终有效评论.xlsx')

data[['review']]= data[['内容']].values.astype(str)
#data['ProcessedText_end']=data[['ProcessedText_end']].values.astype(str)
data["time"] = data["Episode"]
#data[['review', 'ProcessedText_end', 'time']]
data[['review','time']]

,review,time
0,@麻辣犹豫徐. 这就是我听的小说 挺好听的,1
1,我也想穿越洪荒境界无量，不老不死，不灭，,1
2,易昉脸上疤痕有的时候在左边有的时候在右这也太假了,1
3,这是我第一次看完的唯一短剧，导演历害，这才是短剧的未来。,1
4,我看的这个小说咋还在更新，这里都拍成短剧了哦,1
...,...,...
5889,这部剧的水平也太高了吧！那些剧情重复弱智无脑粗制滥造的短剧好好学学吧！与其粗制滥造10部烂片...,91
5890,导演一定是奇人，这部剧扑朔迷离，跌汤起伏，看的身临其境，感谢导演带来这么好的一部剧,91
5891,这是我熬夜追完的第一部短剧，最后一集了还没看过瘾呢,91
5892,醒酒汤配方给一个呗，两小勺就啥事没有了,91


In [6]:
import pandas as pd
import re

def preprocess_input(text):
    """简单的预处理函数，去除可能引起敏感内容检测的字符或词语"""
    # 使用正则表达式去除特殊字符
    clean_text = re.sub(r'[^\w\s]', '', text)
    return clean_text

# 创建新列存储输出结果
data['quality_features_initial'] = ""

# 循环遍历每个评论
for i, question in enumerate(data['review'][:20]):
    try:
        # 预处理输入
        question = preprocess_input(question)
        
        # 获取相关文档
        docs = db.similarity_search(question, k=3)
        related_docs = get_related_content(docs)
        print(f"Related Docs for question {i+1}: {related_docs}")
        
        # 生成 prompt
        my_prompt = define_prompt(related_docs)
        
        # 清空 outputs 列表
        outputs = []
        
        # 切分评论为较短的片段
        input_segments = [question[i:i+2000] for i in range(0, len(question), 2000)]
        
        # 对每个片段调用大语言模型
        for segment in input_segments:
            model_input = my_prompt + segment
            
            output = llm.invoke(model_input).content
            outputs.append(output)
        
        # 将输出结果添加到新列中
        data.at[i, 'quality_features'] = outputs
        
        # 每处理50条保存一次
        if (i + 1) % 5 == 0:
            # 构造保存文件名
            file_name = f"output_{i + 1}.xlsx"
            # 保存数据到 Excel 文件
            data.to_excel("./02RAG_质量特征匹配阶段/RAG代码/01输出数据/桃花马上请长缨-gpt3.5_置信度.xlsx", index=False)
            # 输出保存成功信息
            print(f"已保存前 {i + 1} 条评论到文件: {file_name}")
    
    except Exception as e:
        print(f"处理第 {i+1} 条评论时出错: {e}")

# 输出结果
print(data[['review', 'quality_features']])


Related Docs for question 1: 评论内容：@麻辣犹豫徐. 这就是我听的小说 挺好听的 。质量特征匹配结果为： 剧情合理性|积极|0.80。
评论内容：我说怎么这么熟悉的名字。我最听在听这个广播剧。 。质量特征匹配结果为： nan。
评论内容：咦 正追的连载小说变成了短剧@贝塔🐮 。质量特征匹配结果为： nan。
评论内容：@纠结症患者选择恐惧症晚期 这是我之前追的一部小说 挺好看的 拍短剧了女主好好看 男主有点潦草 。质量特征匹配结果为： 角色表现|积极|0.77。
my_prompt-->
          你将收到一组微短剧的在线评论文档。你的任务是:识别每个文档中涉及的质量特征(最多3个);然后,进行情感分析,判断情感极性（积极/中立/消极）并计算情感得分；最后,以指定格式输出每个文档对应的质量特征标签、情感级性和情感得分。

          现有的质量特征标签为：文化深度、剧情合理性、叙事节奏、角色表现、动作场面呈现、服化道匹配度、台词质量、音效与配乐。其具体含义如下：
          文化深度：微短剧是否具备思想性和文化价值，能否引发观众共鸣或思考，并传递积极的社会价值观。
          剧情合理性：剧情逻辑是否严谨，故事情节是否连贯、完整，角色行为是否符合设定。
          叙事节奏：故事节奏是否流畅，是否存在冗长或过快的节奏变化影响观感。
          角色表现：演员表演是否真实自然，能否准确表达角色的个性、情感与成长。
          动作场面呈现：动作场面的特效表现是否真实，武术指导是否到位，是否增强视觉冲击力。
          服化道匹配度：服装、化妆和道具是否符合角色设定、时代背景和剧情需求，是否增强角色表现力。
          台词质量：台词是否符合角色设定，语言表达是否自然流畅，避免刻意生硬。
          音效与配乐：背景音乐、音效是否恰当，能否有效增强剧情氛围和情感表达。

          情感级性包括：积极、中立和消极。积极对应的情感得分取值范围为(0,1],中立为0,消极为[-1,0)。
            
          
          [示例]
          示例 1:将"叙事节奏"指定给文档，并给出质量特征标签对应的情感级性和情

##相似度验证

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import re
import os
from tqdm import tqdm  # 添加进度条

# 设置工作目录
os.chdir("D:/研究生论文/小论文2/代码")

# 初始化模型
model = SentenceTransformer('./02RAG_质量特征匹配阶段/RAG代码/embedding model/m3e-base')

# 质量特征描述
FEATURE_DESCRIPTIONS = {
    "文化深度": "微短剧是否具备思想性和文化价值，能否引发观众共鸣或思考，并传递积极的社会价值观。",
    "剧情合理性": "剧情逻辑是否严谨，故事情节是否连贯、完整，角色行为是否符合设定。",
    "叙事节奏": "故事节奏是否流畅，是否存在冗长或过快的节奏变化影响观感。",
    "角色表现": "演员表演是否真实自然，能否准确表达角色的个性、情感与成长。",
    "动作场面呈现": "动作场面的特效表现是否真实，武术指导是否到位，是否增强视觉冲击力。",
    "服化道匹配度": "服装、化妆和道具是否符合角色设定、时代背景和剧情需求，是否增强角色表现力。",
    "台词质量": "台词是否符合角色设定，语言表达是否自然流畅，避免刻意生硬。",
    "音效与配乐": "背景音乐、音效是否恰当，能否有效增强剧情氛围和情感表达。"
}

def compute_confidence(text, feature_name):
    """计算评论与特征的相似度置信度"""
    try:
        desc = FEATURE_DESCRIPTIONS.get(feature_name, "")
        if not desc:
            return 0.0
        
        # 编码文本和特征描述
        text_emb = model.encode(text, convert_to_tensor=True)
        feat_emb = model.encode(desc, convert_to_tensor=True)
        
        # 计算余弦相似度
        similarity = cosine_similarity(text_emb.reshape(1, -1), feat_emb.reshape(1, -1))
        return float(similarity[0][0])
    except Exception as e:
        print(f"计算置信度时出错 - 特征: {feature_name}, 错误: {str(e)}")
        return 0.0

def parse_features(qf_str):
    """解析quality_features列的多特征情况"""
    if pd.isna(qf_str) or qf_str.strip() in ["['None']", "None", ""]:
        return []
    
    try:
        # 尝试解析为列表
        if qf_str.startswith('[') and qf_str.endswith(']'):
            features_list = ast.literal_eval(qf_str)
        else:
            features_list = [qf_str]
        
        features = []
        for item in features_list:
            # 清理字符串
            item = str(item).strip(" '\"")
            if not item or item == "None":
                continue
                
            # 分割特征信息
            parts = [p.strip() for p in item.split('|') if p.strip()]
            if len(parts) == 3:
                try:
                    features.append({
                        'feature': parts[0],
                        'polarity': parts[1],
                        'score': float(parts[2])
                    })
                except ValueError:
                    continue
        return features
    except Exception as e:
        print(f"解析特征时出错: {qf_str}, 错误: {str(e)}")
        return []

def process_file(input_path, high_conf_output, low_conf_output):
    """处理文件并分流保存结果"""
    try:
        # 读取Excel文件
        df = pd.read_excel(input_path)
        print(f"成功读取文件: {input_path}, 共 {len(df)} 条数据")
        
        # 准备结果容器
        high_conf_data = []
        low_conf_data = []
        
        # 使用tqdm添加进度条
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="处理进度"):
            try:
                review = str(row['review']) if pd.notna(row['review']) else ""
                qf_str = str(row['quality_features']) if pd.notna(row['quality_features']) else ""
                
                features = parse_features(qf_str)
                high_conf_features = []
                low_conf_features = []
                
                for feat in features:
                    try:
                        conf = compute_confidence(review, feat['feature'])
                        record = {
                            'feature': feat['feature'],
                            'polarity': feat['polarity'],
                            'score': feat['score'],
                            'confidence': conf
                        }
                        
                        if conf >= 0.7:
                            high_conf_features.append(record)
                        else:
                            low_conf_features.append(record)
                    except Exception as e:
                        print(f"处理特征时出错 - 行 {idx+1}, 特征: {feat}, 错误: {str(e)}")
                        continue
                
                # 添加到对应结果集
                if high_conf_features:
                    high_conf_data.append({
                        'original_review': review,
                        'original_features': qf_str,
                        'validated_features': "; ".join(
                            [f"{f['feature']}|{f['polarity']}|{f['score']:.2f}|{f['confidence']:.2f}" 
                             for f in high_conf_features]
                        ),
                        'feature_count': len(high_conf_features)
                    })
                
                if low_conf_features:
                    low_conf_data.append({
                        'original_review': review,
                        'original_features': qf_str,
                        'filtered_features': "; ".join(
                            [f"{f['feature']}|{f['polarity']}|{f['score']:.2f}|{f['confidence']:.2f}" 
                             for f in low_conf_features]
                        ),
                        'max_confidence': max([f['confidence'] for f in low_conf_features], default=0),
                        'feature_count': len(low_conf_features)
                    })
                
            except Exception as e:
                print(f"处理行 {idx+1} 时发生严重错误: {str(e)}")
                continue
        
        # 保存高置信度结果
        if high_conf_data:
            high_conf_df = pd.DataFrame(high_conf_data)
            high_conf_df.to_excel(high_conf_output, index=False)
            print(f"已保存高置信度结果 ({len(high_conf_df)} 条) 到: {high_conf_output}")
        else:
            print("警告: 没有高置信度数据需要保存")
        
        # 保存低置信度结果
        if low_conf_data:
            low_conf_df = pd.DataFrame(low_conf_data)
            low_conf_df['filter_reason'] = 'confidence < 0.7'
            low_conf_df.to_excel(low_conf_output, index=False)
            print(f"已保存低置信度结果 ({len(low_conf_df)} 条) 到: {low_conf_output}")
        else:
            print("警告: 没有低置信度数据需要保存")
            
    except Exception as e:
        print(f"处理文件时发生严重错误: {str(e)}")
        raise

if __name__ == "__main__":
    # 输入输出文件路径
    input_file = "./02RAG_质量特征匹配阶段/RAG代码/01输出数据/桃花马上请长缨-gpt3.5.xlsx"
    high_conf_file = "./02RAG_质量特征匹配阶段/RAG代码/01输出数据/桃花马上请长缨_高置信度结果.xlsx"
    low_conf_file = "./02RAG_质量特征匹配阶段/RAG代码/01输出数据/桃花马上请长缨_低置信度结果.xlsx"
    
    # 运行处理
    process_file(input_file, high_conf_file, low_conf_file)